In [23]:
import pandas as pd
import re
import sys
import os

# Añadir la ruta del directorio data_utils al sys.path
# Agregar la carpeta raíz 'data_utils' al sys.path
sys.path.append(os.path.abspath('../'))


In [24]:
from data_utils.configuraciones.variables import variables_data
from data_utils.manipulacion_de_datos.limpieza_y_maniuplacion_de_datos import procesar_descripcion, estandarizado_columnas
from data_utils.funciones_de_busqueda.busqueda_por_filtro import filtrar_por_laboratorio
from data_utils.funciones_de_busqueda.busquedas_por_texto import main_principal

Instanciamientos de Clases

In [25]:
variables_data = variables_data()

CARGA DE DATOS DE CSV

Carga de CSV Master Productos

In [26]:
#Carga de Producto
df_Producto = pd.read_csv(variables_data.csv_producto)

# Eliminar filas donde todas sus columnas están vacías
df_MasterProductos = df_Producto.dropna(how='all').copy()

# Eliminar columnas donde todos sus valores están vacíos
df_MasterProductos = df_MasterProductos.dropna(axis=1, how='all')

df_MasterProductos[variables_data.descripcion_limpia_MasterProductos] = df_MasterProductos[variables_data.name_columna_1_MasterProductos].fillna('') + " " + \
                                                                        df_MasterProductos[variables_data.name_columna_2_MasterProductos].fillna('')
#Indexacion 
df_MasterProductos[variables_data.index_productos] = df_MasterProductos.index
df_MasterProductos.columns

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_7060\1147232153.py:2: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_Producto = pd.read_csv(variables_data.csv_producto)


Index(['niprod', 'cod_prod', 'cod_abrev', 'descrip1', 'descrip2', 'abrev',
       'cod_catprod', 'codbarra', 'cod_fbarra', 'unid_v', 'base_v', 'precio',
       'mon_v', 'costo', 'base_c', 'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c',
       'unid_alt', 'rel_c_a', 'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif',
       'pped', 'st_min', 'st_max', 'observ', 'pc_comision', 'pc_utilidad',
       'ind_stock', 'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto',
       'ninum', 'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item',
       'cant_rep', 'pto_rep', 'atrib0', 'atrib1', 'atrib2',
       'descripcion_limpia_producto', 'index_producto'],
      dtype='object')

Carga de CSV Proveedor

In [27]:
#Carga Proveedor
#variables_data.cvs_proveedor
cvs_proveedor = 'Phoenix.csv'
df_Proveedor = pd.read_csv(cvs_proveedor)

# Eliminar filas donde todas sus columnas están vacías
df_Proveedor = df_Proveedor.dropna(how='all')
df_Proveedor = df_Proveedor.dropna(axis=1, how='all')

# Eliminar columnas donde todos sus valores están vacíos
df_Proveedor = df_Proveedor.dropna(axis=1, how='all')
#Indexacion 
df_Proveedor[variables_data.index_proveedor] = df_Proveedor.index

Estatandarizacion de Columnas data Proveedor

In [28]:
df_Proveedor = estandarizado_columnas(
    df_Proveedor,
    variables_data.name_columna_1_Proveedor,
    variables_data.name_columna_2_Proveedor,
    variables_data.descripcion_limpia_Proveedor
)
df_Proveedor

,publicado/\nno publicado,u n,linea,ean,cod. dispro,cod. elea,cod. cid latina,registro alfabeta,codigo kairos,pami,certif nro,descripcion,presentacion,index_proveedor,descripcion_limpia_proveedor
0,PUBLICADO,BIO,Biotecnología,7796285049928,14954,24992,2000601.0,27613,NaN,NaN,48650,3TC + AZT ELEA,Env. x 60 comprimidos,0,3TC + AZT ELEA Env. x 60 comprimidos
1,PUBLICADO,BIO,Biotecnología,7796285050658,17100,25065,2003701.0,32410,NaN,NaN,49022,ABACAVIR ELEA,Env. x 60 comprimidos recub.,1,ABACAVIR ELEA Env. x 60 comprimidos recub.
2,PUBLICADO,BIO,Biotecnología,7796285283551,19341,28355,NaN,56653,NaN,NaN,58983,ALCAF,60mg F. Amp. Polvo x 1,2,ALCAF 60mg F. Amp. Polvo x 1
3,PUBLICADO,BIO,Biotecnología,7796285283520,19869,28352,NaN,60305,NaN,NaN,NaN,BENJOR,50 mg x comp x 30,3,BENJOR 50 mg x comp x 30
4,PUBLICADO,BIO,Biotecnología,7796285277833,17779,27783,NaN,53125,NaN,NaN,57934,BEVAX 100 MG,Env x 1 vial de 4 ml.,4,BEVAX 100 MG Env x 1 vial de 4 ml.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,PUBLICADO,P.C.,Cardio,7796285284442,19431,28444,9008752.0,57299,283302.0,658271-2,NaN,ZATIUM MET,50/500 MG COMP X 60,697,ZATIUM MET 50/500 MG COMP X 60
698,PUBLICADO,P.C.,Cardio,7796285284459,19434,28445,9008750.0,57298,283303.0,658284-1,NaN,ZATIUM MET,50/850 MG COMP X 30,698,ZATIUM MET 50/850 MG COMP X 30
699,PUBLICADO,P.C.,Cardio,7796285284473,19430,28447,9008753.0,57297,283304.0,658284-2,NaN,ZATIUM MET,50/850 MG COMP X 60,699,ZATIUM MET 50/850 MG COMP X 60
700,PUBLICADO,P.C.,Visual,7796285255169,13702,25516,3628302.0,36159,165772.0,53058410,51699,ZOPIROL,0.50% SOL.OFT. x 5 ML,700,ZOPIROL 0.50% SOL.OFT. x 5 ML


Estandarizacion de COLUMNAS

In [29]:
# Procesamiento de df_MasterProductos: descripcion_limpia_producto
df_MasterProductos = procesar_descripcion(df_MasterProductos, variables_data.descripcion_limpia_MasterProductos)
# Procesamiento de df_Proveedor: descripcion_limpia_proveedor
df_Proveedor = procesar_descripcion(df_Proveedor, variables_data.descripcion_limpia_Proveedor)

In [36]:
# Pedir al usuario que ingrese el nombre del laboratorio
laboratorio_input = input("Ingresa el nombre del laboratorio: ").strip()  # El usuario ingresa el nombre del laboratorio

# Filtrar productos por laboratorio
df_MasterProductos_Filtro = filtrar_por_laboratorio(df_MasterProductos, laboratorio_input)


df_resultado_similiridad = main_principal(df_MasterProductos_Filtro, df_Proveedor)
df_resultado_similiridad.columns

Se encontraron 402 productos coincidentes del laboratorio 'Phoenix'.


Index(['similaridad', 'publicado/\nno publicado_x', 'u n_x', 'linea_x',
       'ean_x', 'cod. dispro_x', 'cod. elea_x', 'cod. cid latina_x',
       'registro alfabeta_x', 'codigo kairos_x', 'pami_x', 'certif nro_x',
       'descripcion_x', 'presentacion_x', 'index_proveedor_x',
       'descripcion_limpia_proveedor', 'niprod', 'cod_prod', 'cod_abrev',
       'descrip1', 'descrip2', 'abrev', 'cod_catprod', 'codbarra',
       'cod_fbarra', 'unid_v', 'base_v', 'precio', 'mon_v', 'costo', 'base_c',
       'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c', 'unid_alt', 'rel_c_a',
       'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif', 'pped', 'st_min',
       'st_max', 'observ', 'pc_comision', 'pc_utilidad', 'ind_stock',
       'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto', 'ninum',
       'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item', 'cant_rep',
       'pto_rep', 'atrib0', 'atrib1', 'atrib2', 'descripcion_limpia_producto',
       'index_producto', 'publicado/\nno publicado_y', 'u 

Input de Laboratorio

Procesamiento para comparacion de  Descripciones

In [38]:
df_resultado_similiridad.columns

,similaridad,niprod,cod_prod,descripcion_limpia_proveedor,descripcion_limpia_producto
0,0.154652,91047.0,41201,3 tc azt elea envase x 60 comprimidos,fingras 60 miligramos capsulas x 60
1,0.165141,91047.0,41201,abacavir elea envase x 60 comprimidos recub,fingras 60 miligramos capsulas x 60
2,0.212607,91018.0,37191,alcaf 60 miligramos f ampolla polvo x 1,doxtran b 12 inyeccion frasco ampolla x 3
3,0.290799,26498.0,36926,benjor 50 miligramos x comprimidos x 30,causalon comprimidos x 50
4,0.333056,91318.0,39980,beva x 100 miligramos envase x 1 vial de 4 mil...,xeomin 100 ui vial x 1
...,...,...,...,...,...
697,0.210294,91047.0,41201,zatium met 50500 miligramos comprimidos x 60,fingras 60 miligramos capsulas x 60
698,0.200132,91224.0,26438,zatium met 50850 miligramos comprimidos x 30,pioglit 30 miligramos comprimidos x 30
699,0.213503,91047.0,41201,zatium met 50850 miligramos comprimidos x 60,fingras 60 miligramos capsulas x 60
700,0.450756,90987.0,38577,zopirol 050% solucion oftalmica x 5 mililitros,clare x compuesto solucion oftalmica x 5 milil...


In [50]:
df_resultado_similiridad[df_resultado_similiridad['similaridad'] > 0.8][['similaridad', 'niprod', 'cod_prod', 'descripcion_limpia_proveedor', 'descripcion_limpia_producto', 'index_proveedor_y','ean_x','codbarra']]

,similaridad,niprod,cod_prod,descripcion_limpia_proveedor,descripcion_limpia_producto,index_proveedor_y,ean_x,codbarra
470,0.938959,91093.0,20310,loreme x descongestivo comprimidos x 5,loreme x descongestivo comprimidos x 10,470,7791848079022,7791848079015
513,0.890747,91160.0,12676,neumote x nebu x 20 mililitros,neumote x nebu envase x 20 mililitros,513,7791848245007,7791848244901.0


INDEX DE SUSTRACCION PARA EL DF PROVEEDOR

In [51]:
#Sutraccion de productos sobre df ORIGINAL DE PROVEEDORES.
similiridad = 'similaridad'

umbral_similaridad = 0.8  # Ajusta este valor según tu caso
df_filtrado_similaridad = df_resultado_similiridad[df_resultado_similiridad[similiridad] > umbral_similaridad]

# Ahora, con los índices originales, extraemos los productos del df_Proveedor
# Filtramos por 'indice_original_proveedor' para obtener solo las filas que superaron el umbral
df_Proveedor_Similiridad_filtrado = df_Proveedor[df_Proveedor[variables_data.index_proveedor].isin(df_filtrado_similaridad['index_proveedor_x'])]

# Mostrar los productos filtrados del proveedor
df_Proveedor_Similiridad_filtrado


,publicado/\nno publicado,u n,linea,ean,cod. dispro,cod. elea,cod. cid latina,registro alfabeta,codigo kairos,pami,certif nro,descripcion,presentacion,index_proveedor,descripcion_limpia_proveedor
470,PUBLICADO,P.C.,Vías Respiratorias,7791848079022,4902,28235,2935001.0,30102,174041.0,NaN,46244,LOREMEX DESCONGESTIVO,Comprimidos x 5,470,loreme x descongestivo comprimidos x 5
513,PUBLICADO,P.C.,Vías Respiratorias,7791848245007,13963,28161,3072501.0,45886,80332.0,3917062,43966,NEUMOTEX NEBU,X 20 ML.,513,neumote x nebu x 20 mililitros


In [54]:
df_MasterProductos.columns

#Arquitectura df_Productos

# Niprod = Sistema Carena
# ean = 
# descrip1 = Descripcion Provedor
# descrip2  = Presentacion Proveedor
# atrib0 = laboratorio
# codbarra  = codbarra

Index(['niprod', 'cod_prod', 'cod_abrev', 'descrip1', 'descrip2', 'abrev',
       'cod_catprod', 'codbarra', 'cod_fbarra', 'unid_v', 'base_v', 'precio',
       'mon_v', 'costo', 'base_c', 'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c',
       'unid_alt', 'rel_c_a', 'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif',
       'pped', 'st_min', 'st_max', 'observ', 'pc_comision', 'pc_utilidad',
       'ind_stock', 'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto',
       'ninum', 'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item',
       'cant_rep', 'pto_rep', 'atrib0', 'atrib1', 'atrib2',
       'descripcion_limpia_producto', 'index_producto'],
      dtype='object')